In [173]:
import pandas as pd
import numpy as np
import collections

In [103]:
path = './data/Population.csv'
data = pd.read_csv(path)

In [104]:
data.dtypes

County            object
Year-Month         int64
Population         int64
Pop Indensity    float64
dtype: object

In [105]:
data.head()

,County,Year-Month,Population,Pop Indensity
0,New Taipei,201001,3876070,1888.4
1,New Taipei,201002,3878282,1889.5
2,New Taipei,201003,3882039,1891.3
3,New Taipei,201004,3884754,1892.6
4,New Taipei,201005,3885507,1893.0


In [107]:
data['Year-Month'] = data['Year-Month'].astype(str)

In [175]:
cty = data.County.unique()
print(cty)

cty_data = collections.defaultdict()

for ct in cty:
    
    cty_data[ct] = data[data['County'] == ct]


['New Taipei' 'Taipei' 'Taoyuan' 'Taichung' 'Tainan' 'Kaohsiung' 'Yilan'
 'Hsinchu county' 'Miaoli' 'Changhua' 'Nantou' 'Yunlin' 'Chiayi county'
 'Pingtung' 'Taitung' 'Hualien' 'Penghu' 'Keelung' 'Hsinchu' 'Chiayi city'
 'Kinmen' 'Lienchang']


In [280]:
cty_data[cty[5]].tail()

,County,Year-Month,Population,Pop Indensity
673,Kaohsiung,201901,2773607,993.2
674,Kaohsiung,201902,2773596,993.2
675,Kaohsiung,201903,2773271,993.1
676,Kaohsiung,201904,2773229,993.0
677,Kaohsiung,201905,2773026,993.0


In [271]:
print("Create nested dict...")

cty_yyyymm_pop_popid = collections.defaultdict(dict)


for ct in tqdm(cty):
    
    yyyymm_pop_popid = collections.defaultdict(dict)

    for ym, p, pi in zip(cty_data[ct]['Year-Month'], cty_data[ct]['Population'], cty_data[ct]['Pop Indensity']):
        
        yyyymm_pop_popid[ym] = (p, pi) # population, pop intensity 
        
    cty_yyyymm_pop_popid[ct] = yyyymm_pop_popid

100%|██████████| 22/22 [00:00<00:00, 4833.41it/s]

Create nested dict...


In [281]:
cty_yyyymm_pop_popid['Tainan']['201905']

(1882624, 858.9)

In [282]:
# add 201906 pop, pop density
for ct in cty_yyyymm_pop_popid.keys():
    
    p, pi = cty_yyyymm_pop_popid[ct]['201904']
    
    p1, pi1 = cty_yyyymm_pop_popid[ct]['201905']
    
    p2 = (p + p1)//2
    
    pi2 = (pi + pi1)/2
    
    cty_yyyymm_pop_popid[ct]['201906'] = (p2, pi2)


In [283]:
def power_beta(current_month, next_month):
    return (next_month/current_month)**(1/4)


In [294]:
#from datetime import date,timedelta
from tqdm import tqdm
import collections

df = pd.DataFrame({'INSP_DATE2':pd.date_range(start = '2012-01-01', end = '2019-06-20')})

a =  df['INSP_DATE2'] - pd.offsets.Week(weekday=6)
b =  df['INSP_DATE2'] + pd.offsets.Week(weekday=5)

m1 = df['INSP_DATE2'] != (a + pd.offsets.Week())
m2 = df['INSP_DATE2'] != (b - pd.offsets.Week())

df['c'] = df['INSP_DATE2'].mask(m1, a)
df['d'] = df['INSP_DATE2'].mask(m2, b)
df.shape

(2728, 3)

In [298]:

rows = []
w = 1 # start week
y = 2012 # start year

# generate standard yyyy_ww

for b in tqdm(range(0, df.shape[0], 7)):

    row = []
    win = df.iloc[b : (b+7), :]
    date = win.c
    #print(str(date).split('-'))
    #y, _, _ = str(list(date)[0]).split('-')
    
       
    if w <10:
        w_str = '0' + str(w)
    else:
        w_str = str(w)
    win["Year_Week"] = str(y) + '_' + str(w_str)
    
    if y != 2014 and w == 52:
        w = 0
        y += 1
    elif y == 2014 and w == 53:
        w = 0
        y += 1
    w += 1
    
    rows.append(win)
    
# cancat  
year_weekly = pd.concat(rows)

# unique week
week_total = year_weekly.Year_Week.unique()

# yyyy_mm_dd: yyyy_ww
date_week = collections.defaultdict(list)

for i in range(year_weekly.shape[0]):
    
    date_key = year_weekly.INSP_DATE2[i]
    
    k = str(date_key).split(' ')[0]
    yyyy, mm, dd = k.split('-')
    yyyymm = yyyy+mm
    week_val = year_weekly.Year_Week[i]
    
    # yyyymm: yyyy_ww
    date_week[yyyymm].append(week_val)

  0%|          | 0/390 [00:00<?, ?it/s]/home/basic/tensorflow/tensorflow_pkg/tf_1.12_cuda10.0/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 390/390 [00:08<00:00, 44.24it/s]


In [302]:
np.unique(date_week['201201'])

array(['2012_01', '2012_02', '2012_03', '2012_04', '2012_05'], dtype='<U7')

In [276]:
np.unique(date_week['201303'])

array(['2013_10', '2013_11', '2013_12', '2013_13', '2013_14'], dtype='<U7')

In [303]:
yyyymm_yyyyww = collections.defaultdict()

for yyyymm, list_yyyyww in date_week.items():
    
    yyyymm_yyyyww[yyyymm] = np.unique(list_yyyyww)

In [304]:
yyyymm_yyyyww['201303']

array(['2013_09', '2013_10', '2013_11', '2013_12', '2013_13', '2013_14'],
      dtype='<U7')

In [305]:
cty_yyyymm_pop_popid['Hualien']

defaultdict(dict,
            {'201001': (341026, 73.7),
             '201002': (340997, 73.7),
             '201003': (340794, 73.6),
             '201004': (340705, 73.6),
             '201005': (340423, 73.5),
             '201006': (339671, 73.4),
             '201007': (339659, 73.4),
             '201008': (339546, 73.4),
             '201009': (339295, 73.3),
             '201010': (339092, 73.3),
             '201011': (338961, 73.2),
             '201012': (338805, 73.2),
             '201101': (338634, 73.2),
             '201102': (338481, 73.1),
             '201103': (338276, 73.1),
             '201104': (338048, 73.0),
             '201105': (337758, 73.0),
             '201106': (337557, 72.9),
             '201107': (337382, 72.9),
             '201108': (337193, 72.9),
             '201109': (336978, 72.8),
             '201110': (336885, 72.8),
             '201111': (336836, 72.8),
             '201112': (336838, 72.8),
             '201201': (336733, 72.8),
       

In [307]:
print("Create nested dict...")

cty_yyyymm_ww_pop_popid = collections.defaultdict(dict)


yyyymm_ww_pop_popid = collections.defaultdict(dict)

for ct in tqdm(cty):
    
    yyyymm_list = sorted(cty_yyyymm_pop_popid[ct].keys())
    
    print('\n', ct)
    
    
    yyyyww_new_pop_popid = collections.defaultdict()

    for i in range(len(yyyymm_list)):
        
        yyyymm = yyyymm_list[i]
        
        inext = i + 1
        
        if inext == len(yyyymm_list):
            continue
            
        next_yyyymm = yyyymm_list[(i + 1)]
        
        pop, popid = cty_yyyymm_pop_popid[ct][yyyymm]
        
        #print('\npop, popid ', pop, popid)
        
        next_pop, next_popid = cty_yyyymm_pop_popid[ct][next_yyyymm]
        
        a = power_beta(pop, next_pop)
        
        #print('\na = ', a)
        
        b = power_beta(popid, next_popid)
        try:
            list_yyyyww = yyyymm_yyyyww[yyyymm]
        except: KeyError
        #print('\n', list_yyyyww)
        
        

        for j in range(len(list_yyyyww)):
            
            yyyyww = list_yyyyww[j]
            
            alp = j + 1
            
            new_pop = pop * (a**alp)
            #print('\na**alp = ', a**alp)

            new_popid = popid * (b**alp)
            
            #print('\new_pop, new_popid = ', new_pop, new_popid)
            yyyyww_new_pop_popid[yyyyww] = (new_pop, new_popid)
            
        yyyymm_ww_pop_popid[yyyymm] = yyyyww_new_pop_popid
        
    cty_yyyymm_ww_pop_popid[ct] = yyyyww_new_pop_popid
        


100%|██████████| 22/22 [00:00<00:00, 1691.56it/s]

Create nested dict...

 New Taipei

 Taipei

 Taoyuan

 Taichung

 Tainan

 Kaohsiung

 Yilan

 Hsinchu county

 Miaoli

 Changhua

 Nantou

 Yunlin

 Chiayi county

 Pingtung

 Taitung

 Hualien

 Penghu

 Keelung

 Hsinchu

 Chiayi city

 Kinmen

 Lienchang


In [308]:
cty_yyyymm_ww_pop_popid['Hualien']

defaultdict(None,
            {'2012_01': (336704.99650694104, 72.77498711192429),
             '2012_02': (336676.99534271716, 72.74998281786739),
             '2012_03': (336648.9965071347, 72.72498711487655),
             '2012_04': (336621.0, 72.70000000000002),
             '2012_05': (336586.99484759185, 72.7),
             '2012_06': (336552.9931303538, 72.7),
             '2012_07': (336518.9948479388, 72.7),
             '2012_08': (336484.9999999999, 72.7),
             '2012_09': (336443.2422274367, 72.7),
             '2012_10': (336401.48963701096, 72.7),
             '2012_11': (336359.74222807976, 72.7),
             '2012_12': (336318.0, 72.7),
             '2012_13': (336276.2629521288, 72.7),
             '2012_14': (336257.483668193, 72.6749870941823),
             '2012_15': (336196.9782255635, 72.64998279421681),
             '2012_16': (336136.4836701522, 72.62498709714268),
             '2012_17': (336076.00000000006, 72.6),
             '2012_18': (336061.999125

In [309]:
tmp2 = []
for ct in cty:
    yyyyww_pop_pi = cty_yyyymm_ww_pop_popid[ct]
    for yyyyww in sorted(yyyyww_pop_pi.keys()):
        p, pi = yyyyww_pop_pi[yyyyww]
        tmp = [ct, yyyyww, p, pi]
        tmp2.append(tmp)
        
fin_data = pd.DataFrame(tmp2, columns = ['County', 'YYYYWW', 'Population', 'Pop Density'])
fin_data.head()
fin_data.to_csv('./data/population_tw_weekly.csv')